# 16. Logistic Regression (분류)

## 설정

### 설정

In [14]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd

import pingouin as pg
from scipy import stats
import statsmodels.api as sm

from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices

import statsmodels.stats.api as sms
from statsmodels.compat import lzip

In [2]:
!apt-get install -y fonts-nanum
!fc-chache -fv
!rm ~/.cache/matplotlib -rf

'apt-get'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.
'fc-chache'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.
'rm'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [3]:
sns.set_theme(style="darkgrid")
plt.rc('font', family = 'Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False

### 데이터 불러오기 + 데이터 전처리

In [22]:
df = pd.read_csv('./data/14_1.LR.csv', encoding = 'cp949')
df.head()

,id,진단,반지름,질감,주변부,크기,평활도,조밀성,오목,대칭,프랙탈
0,1,1,18.0,10.4,122.8,1001.0,0.118,0.278,0.300,0.242,0.079
1,2,1,20.6,17.8,132.9,1326.0,0.085,0.079,0.087,0.181,0.057
2,3,1,19.7,21.3,130.0,1203.0,0.110,0.160,0.197,0.207,0.060
3,4,0,13.5,14.4,87.5,566.3,0.098,0.081,0.067,0.189,0.058
4,5,0,13.1,15.7,85.6,520.0,0.108,0.127,0.046,0.197,0.068


In [5]:
# 행, 열 개수 세기

df.shape

(565, 11)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 565 entries, 0 to 564
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      565 non-null    int64  
 1   진단      565 non-null    int64  
 2   반지름     565 non-null    float64
 3   질감      565 non-null    float64
 4   주변부     565 non-null    float64
 5   크기      565 non-null    float64
 6   평활도     565 non-null    float64
 7   조밀성     565 non-null    float64
 8   오목      565 non-null    float64
 9   대칭      565 non-null    float64
 10  프랙탈     565 non-null    float64
dtypes: float64(9), int64(2)
memory usage: 48.7 KB


In [7]:
df.describe().round(2).T

,count,mean,std,min,25%,50%,75%,max
id,565.0,283.00,163.25,1.00,142.00,283.00,424.00,565.00
진단,565.0,0.37,0.48,0.00,0.00,0.00,1.00,1.00
반지름,565.0,14.14,3.53,7.00,11.70,13.40,15.90,28.10
질감,565.0,19.29,4.31,9.70,16.20,18.80,21.80,39.30
주변부,565.0,92.03,24.37,43.80,75.20,86.30,104.30,188.50
크기,565.0,655.84,352.94,143.50,419.80,551.10,788.50,2501.00
평활도,565.0,0.10,0.01,0.05,0.09,0.10,0.10,0.16
조밀성,565.0,0.10,0.05,0.02,0.06,0.09,0.13,0.34
오목,565.0,0.09,0.08,0.00,0.03,0.06,0.13,0.43
대칭,565.0,0.18,0.03,0.11,0.16,0.18,0.20,0.30


In [26]:
# 범주형 변수 처리

df['진단'].replace({1:'악성', 0:'양성'}, inplace = True)
df['진단'] = df['진단'].astype('category')

df.head()

C:\Users\user\AppData\Local\Temp\ipykernel_19760\3897784624.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['진단'].replace({1:'악성', 0:'양성'}, inplace = True)


,id,진단,질감,크기,평활도,조밀성,오목,대칭,프랙탈
0,1,악성,10.4,1001.0,0.118,0.278,0.300,0.242,0.079
1,2,악성,17.8,1326.0,0.085,0.079,0.087,0.181,0.057
2,3,악성,21.3,1203.0,0.110,0.160,0.197,0.207,0.060
3,4,양성,14.4,566.3,0.098,0.081,0.067,0.189,0.058
4,5,양성,15.7,520.0,0.108,0.127,0.046,0.197,0.068


In [10]:
df.columns

Index(['id', '진단', '반지름', '질감', '주변부', '크기', '평활도', '조밀성', '오목', '대칭', '프랙탈'], dtype='object')

In [31]:
column = ['질감', '크기', '평활도', '조밀성', '오목', '대칭', '프랙탈'] #독립변수

formula = "진단 ~ " + " + ".join(column)
formula

'진단 ~ 질감 + 크기 + 평활도 + 조밀성 + 오목 + 대칭 + 프랙탈'

In [32]:
y, x = dmatrices(formula,
                 data = df,
                 return_type = 'dataframe') # 종속변수 먼저 작성

In [33]:
# 다중공손성

vif = pd.DataFrame()
vif['VIF Factor'] = [variance_inflation_factor(x.values, i ) for i in range(x.values.shape[1])]
vif['features'] = x.columns

print(vif.round(1))

   VIF Factor   features
0       392.3  Intercept
1         1.2         질감
2         4.7         크기
3         2.2        평활도
4         8.6        조밀성
5         7.4         오목
6         1.8         대칭
7         4.3        프랙탈


VIF 큰 값 제거

In [23]:
df = df.drop(['반지름', '주변부'], axis=1)

In [35]:
df.head()

,id,진단,질감,크기,평활도,조밀성,오목,대칭,프랙탈
0,1,악성,10.4,1001.0,0.118,0.278,0.300,0.242,0.079
1,2,악성,17.8,1326.0,0.085,0.079,0.087,0.181,0.057
2,3,악성,21.3,1203.0,0.110,0.160,0.197,0.207,0.060
3,4,양성,14.4,566.3,0.098,0.081,0.067,0.189,0.058
4,5,양성,15.7,520.0,0.108,0.127,0.046,0.197,0.068


In [34]:
y.head()

,진단[악성],진단[양성]
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0
3,0.0,1.0
4,0.0,1.0


In [36]:
y = y.drop('진단[양성]', axis = 1)  
y.head()

,진단[악성]
0,1.0
1,1.0
2,1.0
3,0.0
4,0.0


In [38]:
model = sm.Logit(y, x)
result = model.fit()

Optimization terminated successfully.
         Current function value: 0.105936
         Iterations 11


In [39]:
print(result.summary())

                           Logit Regression Results                           
Dep. Variable:                 진단[악성]   No. Observations:                  565
Model:                          Logit   Df Residuals:                      557
Method:                           MLE   Df Model:                            7
Date:                Fri, 09 Jan 2026   Pseudo R-squ.:                  0.8390
Time:                        16:41:39   Log-Likelihood:                -59.854
converged:                       True   LL-Null:                       -371.75
Covariance Type:            nonrobust   LLR p-value:                1.835e-130
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -34.9140      7.612     -4.587      0.000     -49.833     -19.995
질감             0.4755      0.080      5.926      0.000       0.318       0.633
크기             0.0170      0.003      5.330      0.0

- 평활도가 가장 큰 영향을 미침
- 오목, 대칭도 영향을 줌